# Node Dependencies

Preparation (already done):

```bash
grep -v 'node_modules' 03_analyses/library_versions/packagejson-files | xargs jq -s '[ .[].devDependencies | . | objects | to_entries | map([.key,.value, "devDependencies",input_filename]) ] | add' 2>/dev/null | jq -sr 'add | .[] | select(all(type=="string")) | @csv' >03_analyses/library_versions/packagejson-deps.csv

grep -v 'node_modules' 03_analyses/library_versions/packagejson-files | xargs jq -s '[ .[].dependencies | . | objects | to_entries | map([.key,.value, "dependencies",input_filename]) ] | add' 2>/dev/null | jq -sr 'add | .[] | select(all(type=="string")) | @csv' >>03_analyses/library_versions/packagejson-deps.csv
```

In [8]:
import pandas as pd

filename = '/home/gjungwirth/data/02_data/03_analyses/library_versions/packagejson-deps.csv'

df = pd.read_csv(filename,names=['name','version','type','filepath'])

In [9]:
dfname = df['name']
#len(dfname.unique())
#dfname.describe()
dfnamestats = df['name'].value_counts().rename_axis('unique_values').reset_index(name='counts')
dfnamestats

,unique_values,counts
0,mocha,6467
1,eslint,2383
2,tap,2274
3,tape,2268
4,istanbul,2061
...,...,...
6260,eslint-plugin-prefer-arrow,1
6261,@types/pluralize,1
6262,react-redux-loading-bar,1
6263,dagre-d3,1


In [10]:
import requests

# contains: either error OR: author, versions, keywords, description, name
def get_lib_info_npm(libname):
    res = requests.get('https://registry.npmjs.org/{}'.format(libname))
    return res.json()

# libinfo = get_lib_info_npm('mocha')

In [11]:
# dfnamestats['libinfo'] = dfnamestats['unique_values'].apply(get_lib_info_npm)
# dfnamestats

storage_file = '/home/gjungwirth/data/02_data/03_analyses/library_versions/node_data_pickle.bak'
#dfnamestats.to_pickle(storage_file)

df2 = pd.read_pickle(storage_file)
#df2.equals(dfnamestats)
dfnamestats = df2

In [12]:
dfnamestats['private_lib'] = dfnamestats['libinfo'].apply(lambda a: 'error' in a.keys())

tmp = dfnamestats[dfnamestats['private_lib'] == True]

#print("\n".join(tmp['unique_values']))

## part 2 - check which kind of version fixation we have

In [13]:
import redf[df.unique_libs.notnull()].unique_libs.values
total = len(df.index)
version_stat = pd.DataFrame(df['version'].apply(lambda x: re.sub('[0-9x.]*$','',str(x))).value_counts().reset_index(name='counts'))
version_stat['percent'] = version_stat['counts'] * 100 / total
version_stat

SyntaxError: invalid syntax (<ipython-input-13-51bc6fbaa907>, line 1)

## probably do the difference thing here too

In [15]:
from Levenshtein import distance
from itertools import product
import numpy as np

lib_names = list(dfnamestats['unique_values'].values)
dist = [distance(*x) for x in product(lib_names, repeat=2)]

In [16]:
dist_df = pd.DataFrame(np.array(dist).reshape(len(lib_names), len(lib_names)), index=lib_names, columns=lib_names)
dist_df['closest_neighbor'] = dist_df.applymap(lambda x: None if x==0 else x).idxmin(axis="columns")

In [17]:
dist_df['dist_of_neighbor'] = dist_df.apply(lambda x: x[x['closest_neighbor']],axis=1)
dist_df[['closest_neighbor','dist_of_neighbor']]

,closest_neighbor,dist_of_neighbor
mocha,matcha,2
eslint,tslint,1
tap,tape,1
tape,tap,1
istanbul,stable,4
...,...,...
csscomb-next,csscomb-core,4
@types/tinycolor2,@types/colors,5
esformatter-semicolon-first,esformatter-semicolons,5
@rails/activestorage,@rails/actioncable,8


In [18]:
libs_w_distances = dfnamestats.set_index('unique_values').join(dist_df[['closest_neighbor','dist_of_neighbor']])

In [19]:
tmp = libs_w_distances[libs_w_distances['private_lib'] == True].sort_values(by='dist_of_neighbor')

# Links

https://github.com/npm/registry/blob/master/docs/REGISTRY-API.md
https://registry.npmjs.org/mocha
